<a href="https://colab.research.google.com/github/Theedon/AIAgentDemos/blob/main/LLMQuickTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install  langchain_experimental langchain_openai langchain_google_community
!pip install --upgrade openai
!pip install -qU httpx

In [2]:
import os
from google.colab import userdata

os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["OPENAI_API_VERSION"] = userdata.get('OPENAI_API_VERSION')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')

In [3]:
# prompt: generate boilerplate langchain code to test pfrom langchain_core.tools import Tool
# from langchain_experimental.utilities import PythonREPL

from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

import dotenv
from langchain_core.tools import StructuredTool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

google_search = StructuredTool.from_function(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

python_repl = PythonREPL()

tools = [
    Tool(
        name="python_repl",
        func=python_repl.run,
        description="A Python shell. Use this to execute python code.",
    ),
    # google_search
]


# Example using the tool
# You would typically integrate this into a larger LangChain application.
# Here we just demonstrate how to use a tool directly
tool_result = python_repl.run("import numpy as np; print(np.random.rand(3,4))")
print(f"Tool Result: {tool_result}")


Tool Result: [[0.74506747 0.38786141 0.16348148 0.45415955]
 [0.98573118 0.12292858 0.57886413 0.87184194]
 [0.53582739 0.96790169 0.13380741 0.9148603 ]]



In [8]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI

from langchain_openai import (
    AzureChatOpenAI,
    AzureOpenAIEmbeddings,
)

# LLM = AzureChatOpenAI(
#     azure_deployment="4o-mini-deploy",
#     # cache=redis_cache,
# )
LLM = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# LLM = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [9]:
from langchain_core.prompts import ChatPromptTemplate

def get_prompt_for_openai_functions_agent():
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant"),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ]
    )
    return prompt

In [10]:
# prompt: write the code to create an agent that uses the pythonREPL tool

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.agents import (
    AgentExecutor,
    # create_openai_functions_agent,
    create_tool_calling_agent,
)

# Initialize the language model
llm = LLM

# Load the tools (including the Python REPL tool)
# tools = tools
prompt = get_prompt_for_openai_functions_agent()
agent = create_tool_calling_agent(
        llm=llm,
        tools=tools,
        prompt=prompt,
    )

#new agent executor


agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,  # noqa: F821
)

In [11]:
# Example interaction with the agent
res = agent_executor.invoke({"input": "can you find the area of a circle with radius 5 "})
print(res)



> Entering new AgentExecutor chain...

Invoking: `python_repl` with `import math

radius = 5
area = math.pi * radius**2
area`


The area of a circle with a radius of 5 is approximately 78.54 square units.

> Finished chain.
{'input': 'can you find the area of a circle with radius 5 ', 'output': 'The area of a circle with a radius of 5 is approximately 78.54 square units.', 'intermediate_steps': [(ToolAgentAction(tool='python_repl', tool_input='import math\n\nradius = 5\narea = math.pi * radius**2\narea', log='\nInvoking: `python_repl` with `import math\n\nradius = 5\narea = math.pi * radius**2\narea`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Skfdc6RGkZwRQqTbVbbRSifM', 'function': {'arguments': '{"__arg1":"import math\\n\\nradius = 5\\narea = math.pi * radius**2\\narea"}', 'name': 'python_repl'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-ff412ba4